## Importando as bibliotecas e pacotes

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
# from sklearn.pipeline import make_pipeline
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import seaborn as sns
import pandas as pd
import numpy as np

## Carregando o dataset

In [ ]:
path = ""
df_dados_agua = pd.read_csv(path, sep = ",")

## Breve resumo do dataset

In [ ]:
df_dados_agua.head()

In [ ]:
df_dados_agua.dtypes()

In [ ]:
df_dados_agua.info()

## Separando as features do target

In [ ]:
X = df_dados_agua.iloc[:, :6].values
y = df_dados_agua.iloc[:, 6].values.reshape(-1, 1)


## Divisão treino-teste

In [ ]:
tamanho_corte = 0.4
semente = 42
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = tamanho_corte, random_state = semente)

## RandomForestClassifier

In [ ]:
modelo_random_forest = RandomForestClassifier()

In [ ]:
dicionario_random_forest = {
    n_estimators                : np.arange(10, 110, 10),# de 10 a 100
    criterion                   : ["gini", "entropy", "log_loss"],
    max_depth                   : np.arange(2, 8, 1),# de 2 a 7
    min_samples_split           : np.arange(2, 11, 1),# de 2 a 10
    min_samples_leaf            : np.arange(1, 6, 1),# de 1 a 5
    max_features                : ["sqrt", "log2", None],
    max_leaf_nodes              : [5, 10, 15, 20, None],
    n_jobs                      : [-1],
    random_state                : [42]
}

## KMeans

In [ ]:
modelo_kmeans = KMeans()

In [ ]:
dicionario_kmeans = {
    n_clusters      : np.arange(3, 17, 2),# de 3 a 15
    init            : ["random"],
    n_init          : [1, 3, 5, 7, 9, "auto"],
    max_iter        : np.arange(10, 1100, 100),# de 100 a 1000
    tol             : np.logspace(-5, -1, 5),# de 1E-5 a 1E-1
    random_state    : [42],
    algorithm       : ["lloyd", "elkan"]
}

## SVC

In [ ]:
modelo_svc = SVC()

In [ ]:
dicionario_svc = {
    C                           : np.logspace(-3, 3, 7),
    kernel                      : ["linear", "poly", "rbf", "sigmoid", "precomputed"],
    degree                      : np.arange(1, 6, 1),
    tol                         : np.logspace(-5, -1, 5),
    max_iter                    : [100, 200, 300, 400, 500, 600, -1],
    decision_function_shape     : ["ovr", "ovo"],
    random_state                :  [42]
}